# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

# Apple 🍎

Ao iniciar nosso projeto de Ciência dos Dados, tinhamos como objetivo aplicar conceitos vistos em sala, como o Classificador Naive Bayes, Laplace Smothing, entre outros. Para isso, foi proposto que, com a ajuda do ``Python`` e algumas de suas bibliotecas, fizessemos nosso próprio classificador de sentimentos de alguma empresa, com base em tweets obtidos através de uma API disponibilizada. Com isso escolhemos fazer um classificador de relevância da empresa ``Apple`` ``(relevante / irrelevante)`` na ``linguagem inglesa``, por gerar muita ``ambiguidade entre a fruta e a empresa`` em si, caracterizando um problema provavelmente muito recorrente para recebimento de feedbacks.

### Import de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
ps = PorterStemmer()

### Funções do Classificador

In [2]:

def limpeza_de_dados(treinamento, coluna):
    dados_limpos = []
    for e in treinamento[coluna]:
        limpeza = str(e)
        limpeza = ps.stem(limpeza)
        limpeza = limpeza.lower()
        limpeza = limpeza.replace('\n', '')
        limpeza = limpeza.replace('#', '')
        limpeza = limpeza.replace(':', '')
        limpeza = limpeza.replace(',', '')
        limpeza = limpeza.replace('.', '')
        limpeza = limpeza.replace('/', '')
        limpeza = limpeza.replace('//', '')
        limpeza = limpeza.replace('...', '')
        limpeza = limpeza.replace(';', '')
        limpeza = limpeza.replace('\'', '')
        limpeza = limpeza.replace('rt', '')
        limpeza = limpeza.split()
        dados_limpos.append(limpeza)
        
    return dados_limpos

def contagem_palavras(dados_limpos):
    contagem = {}
    for e in dados_limpos:
        for all in e:
            if all not in contagem:
                contagem[all] = 1
            else:
                contagem[all] += 1
    
    return contagem

def frequencia_das_palavras(contagem, prob_total):
    probabilidade_palavra = {}
    soma = sum(contagem.values())
    for k in contagem:
        probabilidade_palavra[k] = np.log((contagem[k] + 1) / (soma + prob_total))

    return probabilidade_palavra

def calculador_probabilidade(frases_tratadas, frequencia_palavra_positivo, frequencia_palavra_negativo, probabilidade_positivo_total, probabilidade_negativo_total, prob_total):
    resultado = []
    laplace_zero = np.log(1 / prob_total)
    for frase in frases_tratadas:
        conta_positivo = []
        conta_negativo = []
        conta_zero = []
        for palavra in frase:
            #for (k,v), (k2,v2) in zip(d.items(), d2.items())
            for (k), (k2) in zip(frequencia_palavra_positivo, frequencia_palavra_negativo): #loop nas frequencias positivas
                if palavra in k: #se a palavra estiver no dicionario de frequencias positivas
                    conta_positivo.append(frequencia_palavra_positivo[k]) #adicionar a lista conta_positivo a probabilidade da palavra k
                elif palavra in k2:
                    conta_negativo.append(frequencia_palavra_negativo[k2])
                elif palavra not in k and palavra not in k2:
                    conta_zero.append(laplace_zero)
        prob_positivo = sum(conta_positivo) + probabilidade_positivo_total + sum(conta_zero)
        prob_negativo = sum(conta_negativo) + probabilidade_negativo_total + sum(conta_zero)
        if prob_positivo > prob_negativo:
            resultado.append(1)
        else:
            resultado.append(0)

    return resultado

## Treinamento do Classificador

### Leitura / Limpeza / Contagem dos Dados

In [3]:
#Leitura dos daods de treinamento
leitura_excel_treinamento = pd.read_excel('tweets_apple.xlsx', sheet_name='Treinamento')

# Divisao dos dados entre relevante e irrelevante (positivo/negativo)
treinamento_raw_positivo = leitura_excel_treinamento[leitura_excel_treinamento.Relevancia==1]
treinamento_raw_negativo = leitura_excel_treinamento[leitura_excel_treinamento.Relevancia==0]

# Limpeza dos dados (positivo/negativo)
limpeza_positivo = limpeza_de_dados(treinamento_raw_positivo, 'Treinamento')
limpeza_negativo = limpeza_de_dados(treinamento_raw_negativo, 'Treinamento')

# Contagem das palavras (positivo/negativo)
contagem_positivo = contagem_palavras(limpeza_positivo)
contagem_negativo = contagem_palavras(limpeza_negativo)


### Aplicação de Naive Bayes e Laplace Smoothing

In [4]:
# Probabilidades Totais (total / total positiva / total negativa)
prob_total = len(treinamento_raw_positivo) + len(treinamento_raw_negativo)
probabilidade_positivo_total = np.log(len(treinamento_raw_positivo) / (len(treinamento_raw_positivo) + len(treinamento_raw_negativo)))
probabilidade_negativo_total = np.log(len(treinamento_raw_negativo) / (len(treinamento_raw_positivo) + len(treinamento_raw_negativo)))

# Probabilidade de cada palavra dado (positivo / negativo) com laplace_smoothing
frequencia_palavra_positivo = frequencia_das_palavras(contagem_positivo, prob_total)
frequencia_palavra_negativo = frequencia_das_palavras(contagem_negativo, prob_total)

## Teste do Classificador

### Limpeza / Calculo / Classificacao dos Dados

In [5]:
#Leitura dos dados de teste
leitura_excel_teste = pd.read_excel('tweets_apple.xlsx', sheet_name='Teste')

#Limpeza dos dados
teste_tratado = limpeza_de_dados(leitura_excel_teste, 'Teste')
teste_tratado


classificacao = calculador_probabilidade(teste_tratado, frequencia_palavra_positivo,frequencia_palavra_negativo, probabilidade_positivo_total, probabilidade_negativo_total, prob_total)
leitura_excel_teste['Classificador'] = classificacao
leitura_excel_teste

,Teste,Relevancia,Classificador
0,rt @nickireignscom: send us some screenshots o...,0,0
1,rt @vlcker77: in line at mcdonald’s.... shitty...,0,1
2,rt @hotfreestyle: travis scott’s “maria i’m dr...,1,0
3,rt @dailyrapfacts: travis scott’s “maria i’m d...,1,0
4,rt @bynoothername: he looks so genuinely happy...,0,0
5,rt @vlcker77: in line at mcdonald’s.... shitty...,0,1
6,rt @shinobininja: shinobi ninja - what if time...,0,0
7,rt @vlcker77: in line at mcdonald’s.... shitty...,0,1
8,rt @hotfreestyle: travis scott’s “maria i’m dr...,1,0
9,"rt @complexmusic: 🙏🏽🔥 @trvisxx, @justinbieber,...",1,0


___
### Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [6]:
i = 0
for e in range(len(leitura_excel_teste.Relevancia)):
    if leitura_excel_teste.Relevancia[e] == leitura_excel_teste.Classificador[e]:
        i += 1
relacao = i / len(leitura_excel_teste.Relevancia)
relacao

0.25

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Com o fim do projeto, chegamos as seguintes conclusões:

Qualitativamente, podemos dizer que o classificador conseguiu aderir bem a algumas noções de classificação humanas, como a capacidade de analisar frases sem efeitos de sentido (ironia e sarcasmo por exemplo). Além disso, quando posto perspectiva quantitativa, vemos que o classificador alcançou um ``bom desempenho`` na classificação das 200 frases da planilha ``Testes``. Podemos dizer isso pois, a partir da correlação calculada anteriormente, o classificador teve uma taxa de acerto de ``75%``. Isto significa que a cada 10 frases, ele acertava 7 aproximadamente. 

Por frases de ``dupla negação e sarcasmo`` serem tratadas pelo programa ``indiferentemente`` (a palavra ``liked``, por exemplo, pode ter conotação tanto positiva, quanto negativa, mas o algoritmo não consegue detectar seu uso, por não a classificar dependendo de seu contexto na frase), houve um ``decaimento`` em sua ``performace``.

Como plano de expansão do projeto, inicialmente deveriam ser adicionadas a capacidade de identificar efeitos de sentido nas frases classificadas. Isto diminuiria a margem de erro feita pela classificador, tornando-o mais preciso e conciso com os dados reais e classificados pela mão humana. Para isso, seriam necessários descobrir alguns padrões, como o uso de hipérboles, contrastes e uso específico de hashtags, idéia que pode ser mais bem desenvolvida entrando em: https://www.onlinejournal.in/IJIRV3I5/082.pdf (Sarcasm Detection using Combinational Logic and Naive Bayes). Ou poderiam ser aplicados algorítmos já disponíveis na internet (https://github.com/surajr/SarcasmDetection).
Outra perk muito relevante que poderia ser adicionado é um classificador de múltiplas variáveis, que o tornaria mais útil do que apenas positivo/negativo (dependendo de sua função), podendo ser melhor entendida no site: (https://research-it.wharton.upenn.edu/programming/naive-bayes-across-multiple-features/).

Vale ressaltar que para nosso projeto, ``não`` seria ``recomendável`` utilizar uma ``base Testes`` feita pelo classificador como ``alimentação`` do ``algoritmo``. Pois seus dados possivelmente podem ter sido classificados corretamente por uma coincidência ou por outros fatores, que não valeriam em todas as situações, aumentando a imprecisão do classificador. (isso pode decorrer dentre outros, de uma base de dados muito pequena, o método de limpeza dos dados ou do próprio método de classificação, que não leva em consideração a ordem das palavras)

Criado nosso plano de expansão, seria de grande interesse das empresas financiar nosso projeto, devido a sua alta aplicabilidade no mercado, desde classificação de spams à verificação de feedbacks no twitter. Por se tratar de um projeto de pequena escala (inicialmente), um futuro investimento aumentaria as chances de convergirmos nossos esforços para suprir suas necessidades, que poderiam ser desde focar nossas pesquisas em um uso para classificação de ``SPAMS`` por localização geográfica até calcular a probabilidade de um determinado usuário estar usando ``VPN`` em seu site.(também por localização geográfica)